In [ ]:
import mediapipe

In [ ]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("../data/faces/raw_images/robbie_trevor.jpg")

fig = plt.figure(figsize=(8, 8))
plt.axis('off')
plt.imshow(img[:, :, ::-1])
plt.show()


In [ ]:
mp_face_mesh = mediapipe.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

In [ ]:
results = face_mesh.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
landmarks = results.multi_face_landmarks[0]

In [ ]:
face_oval = mp_face_mesh.FACEMESH_FACE_OVAL

import pandas as pd
df = pd.DataFrame(list(face_oval), columns = ["p1", "p2"])

In [ ]:
routes_idx = []

p1 = df.iloc[0]["p1"]
p2 = df.iloc[0]["p2"]

for i in range(0, df.shape[0]):

    #print(p1, p2)

    obj = df[df["p1"] == p2]
    p1 = obj["p1"].values[0]
    p2 = obj["p2"].values[0]

    route_idx = []
    route_idx.append(p1)
    route_idx.append(p2)
    routes_idx.append(route_idx)

# -------------------------------

for route_idx in routes_idx:
    print(f"Draw a line between {route_idx[0]}th landmark point to {route_idx[1]}th landmark point")


In [ ]:

routes = []

for source_idx, target_idx in routes_idx:

    source = landmarks.landmark[source_idx]
    target = landmarks.landmark[target_idx]

    relative_source = (int(img.shape[1] * source.x), int(img.shape[0] * source.y))
    relative_target = (int(img.shape[1] * target.x), int(img.shape[0] * target.y))

    cv2.line(img, relative_source, relative_target, (255, 255, 255), thickness = 3)

    routes.append(relative_source)
    routes.append(relative_target)


In [ ]:
import numpy as np

mask = np.zeros((img.shape[0], img.shape[1]))
mask = cv2.fillConvexPoly(mask, np.array(routes), 1)
mask = mask.astype(bool)

out = np.zeros_like(img)
out[mask] = img[mask]
#
# x_start = 0
# y_start = 0
# x_end = 100
# y_end = 100
#
# # Crop the image
# cropped_image = image[y_start:y_end, x_start:x_end]

fig = plt.figure(figsize = (9, 10))
plt.axis('off')
plt.imshow(out[:, :, ::-1])